Importing Basics Modules 

In [1]:
import os
import cv2
import imghdr

C:\Users\Admin\AppData\Local\Temp\ipykernel_10040\2300294395.py:3: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [ ]:
import tensorflow as tf

In [ ]:
data_dir = 'data'
image_exts = ['jpeg','jpg', 'bmp', 'png']

Cleaning the Data (Deleting Unreadable extension/File)

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                #os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            #os.remove(image_path)

Importing Extra Modules

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

Loading Data to Tensorflow

To know info about the module -- module name ??

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data',batch_size=32)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[1].size

In [ ]:
#batch[0]-to store images as numpy arrays 
batch[0]

In [ ]:
#batch[1]-to store labels
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=10, figsize=(20,20))
for idx, img in enumerate(batch[0][:10]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])
#class 0 - Not Leaf 
#class 1 - Early Spot
#class 2 - late spot
#class 3 - Rust 
#class 4 - Normal 

Scaleing data

In [ ]:
data=data.map(lambda x,y:(x/255,y))

In [ ]:
scaled_data= data.as_numpy_iterator()

In [ ]:
batch = scaled_data.next()
batch[0].shape

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

#class 0 - Not Leaf 
#class 1 - Early Spot
#class 2 - late spot
#class 3 - Rust 
#class 4 - Normal 

Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.15)+1
test_size = int(len(data)*.05)+1

In [ ]:
train_size+val_size+test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(Conv2D(32, (3, 3), 1,activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), 1,activation='relu'))
model.add(Conv2D(64, (3, 3), 1,activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), 1,activation='relu'))
model.add(Conv2D(128, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Train

In [ ]:
logdir='traininglogs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    X,y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'presession:{pre.result().numpy()},Recall:{re.result().numpy()},accuracy:{acc.result().numpy()}')

Test

In [ ]:
img = cv2.imread('Early_Leaf_Spot (39).jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
index_min = np.argmax(yhat)

In [ ]:
index_min
#class 0 - Not Leaf 
#class 1 - Early Spot
#class 2 - late spot
#class 3 - Rust 
#class 4 - Normal

In [ ]:
if index_min == 0:
    print('Not Leaf')
elif index_min == 1:
    print('Early Spot')
elif index_min == 2:
    print('Late Spot')
elif index_min == 3:
    print('Rust')
elif index_min == 4:
    print('Normal')    

To Save and load the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','model_6.h5'))

In [ ]:
new_model = load_model(os.path.join('models','model_6.h5'))

In [ ]:
yhat = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat